In [1]:
import lightgbm as lgb
import lightgbm as lgbm
import numpy as np
import pandas as pd
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import mean_squared_log_error
from sklearn.model_selection import train_test_split
from functools import partial
import warnings
from tqdm import tqdm                       # 진행바
from sklearn.metrics import roc_auc_score   # AUC 스코어 계산
from sklearn.metrics import make_scorer
from sklearn.model_selection import KFold   # K-fold CV  
from sklearn.model_selection import cross_validate

In [2]:
train=pd.read_csv('C:\\Users\Hyelin\\bigdata\\FinalProject\\TeamProject\\open data\\train.csv',index_col=0)
test=pd.read_csv('C:\\Users\Hyelin\\bigdata\\FinalProject\\TeamProject\\open data\\test_x.csv',index_col=0)
submission=pd.read_csv('C:\\Users\Hyelin\\bigdata\\FinalProject\\TeamProject\\open data\\sample_submission.csv')
pd.set_option('display.max_columns', 200)

In [3]:
religionMap={
    'Agnostic':0,
    'Atheist':0,
    'Other':0,
    'Buddhist':1,
    'Hindu':1,
    'Muslim':1,
    'Sikh':1,
    'Christian_Catholic':2,
    'Christian_Mormon':2,
    'Christian_Protestant':2,
    'Christian_Other':2,
    'Jewish':2
}

train['religion']=train['religion'].map(religionMap)
test['religion']=test['religion'].map(religionMap)

In [4]:
educationMap={
    0:0,
    1:0,
    2:1,
    3:2,
    4:3
}

train['education']=train['education'].map(educationMap)
test['education']=test['education'].map(educationMap)

In [5]:
raceMap={
    'Asian':2,
    'Arab':3,
    'Black':1,
    'Indigenous Australian':3,
    'Native American':3,
    'Other':3,
    'White':0
}

train['race']=train['race'].map(raceMap)
test['race']=test['race'].map(raceMap)

In [6]:
marriedMap={
    0:0,
    1:0,
    2:1,
    3:2
}

train['married']=train['married'].map(marriedMap)
test['married']=test['married'].map(marriedMap)

In [7]:
def ageBand(x):
    if x=='10s':
        return 0
    elif (x=='20s')or(x=='30s'):
        return 1
    elif (x=='40s')or(x=='50s'):
        return 2
    elif (x=='60s')or(x=='+70s'):
        return 3
    
train['age_group']=train['age_group'].apply(ageBand)
test['age_group']=test['age_group'].apply(ageBand)

In [8]:
Answers = ['QaA', 'QbA', 'QcA', 'QdA', 'QeA',
             'QfA', 'QgA', 'QhA', 'QiA', 'QjA', 
             'QkA', 'QlA', 'QmA', 'QnA', 'QoA', 
             'QpA', 'QqA', 'QrA', 'QsA', 'QtA']
    
flipping_columns = ["QeA", "QfA", "QkA", "QqA", "QrA"]
for flip in flipping_columns: 
    train[flip] = 6 - train[flip]
    test[flip] = 6 - test[flip]
    
flipping_secret_columns = ["QaA", "QdA", "QgA", "QiA", "QnA"]
for flip in flipping_secret_columns: 
    train[flip] = 6 - train[flip]
    test[flip] = 6 - test[flip]
    
train['mach_score'] = train[Answers].mean(axis = 1)
test['mach_score'] = test[Answers].mean(axis = 1)


# 4분위 값
train['mach_per'] = ''
train['mach_per'] = np.where(train["mach_score"] < train["mach_score"].quantile(.25),0,train["mach_per"])
train['mach_per'] = np.where((train["mach_score"]>=train["mach_score"].quantile(.25)) & (train["mach_score"]<=train["mach_score"].quantile(.75)),1,train["mach_per"])
train['mach_per'] = np.where(train["mach_score"] > train["mach_score"].quantile(.75),2,train["mach_per"])                        

test['mach_per'] = ''
test['mach_per'] = np.where(test["mach_score"]<test["mach_score"].quantile(.25),0,test["mach_per"])
test['mach_per'] = np.where((test["mach_score"]>=test["mach_score"].quantile(.25)) & (test["mach_score"]<=test["mach_score"].quantile(.75)),1,test["mach_per"])
test['mach_per'] = np.where(test["mach_score"]>test["mach_score"].quantile(.75),2,test["mach_per"])    

train = train.astype({'mach_per':'int64'})
test = test.astype({'mach_per':'int64'})

In [9]:
Answers = ['QaA', 'QbA', 'QcA', 'QdA', 'QeA',
             'QfA', 'QgA', 'QhA', 'QiA', 'QjA', 
             'QkA', 'QlA', 'QmA', 'QnA', 'QoA', 
             'QpA', 'QqA', 'QrA', 'QsA', 'QtA']
    
flipping_columns = ["QeA", "QfA", "QkA", "QqA", "QrA"]
for flip in flipping_columns: 
    train[flip] = 6 - train[flip]
    test[flip] = 6 - test[flip]
    
flipping_secret_columns = ["QaA", "QdA", "QgA", "QiA", "QnA"]
for flip in flipping_secret_columns: 
    train[flip] = 6 - train[flip]
    test[flip] = 6 - test[flip]
    
train['mach_score'] = train[Answers].mean(axis = 1)
test['mach_score'] = test[Answers].mean(axis = 1)


In [10]:
tp_Answers = ['tp01', 'tp02', 'tp03', 'tp04', 'tp05', 'tp06', 'tp07', 'tp08', 'tp09', 'tp10']
   
tp_flipping_columns = ['tp02', 'tp04', 'tp06', 'tp08', 'tp10']
for tp_flip in tp_flipping_columns: 
    train[tp_flip] = 7 - train[tp_flip]
    test[tp_flip] = 7 - test[tp_flip]

train['tp_score'] = train[tp_Answers].mean(axis = 1)
test['tp_score'] = test[tp_Answers].mean(axis = 1)


In [11]:
tp_Answers = ['tp01', 'tp02', 'tp03', 'tp04', 'tp05', 'tp06', 'tp07', 'tp08', 'tp09', 'tp10']
   
tp_flipping_columns = ['tp02', 'tp04', 'tp06', 'tp08', 'tp10']
for tp_flip in tp_flipping_columns: 
    train[tp_flip] = 7 - train[tp_flip]
    test[tp_flip] = 7 - test[tp_flip]

train['tp_score'] = train[tp_Answers].mean(axis = 1)
test['tp_score'] = test[tp_Answers].mean(axis = 1)

   
# 4분위 값
train['tp_per'] = ''
train['tp_per'] = np.where(train["tp_score"]<train["tp_score"].quantile(.25),0,train["tp_per"])
train['tp_per'] = np.where((train["tp_score"]>=train["tp_score"].quantile(.25)) & (train["tp_score"]<=train["tp_score"].quantile(.75)),1,train["tp_per"])
train['tp_per'] = np.where(train["tp_score"]>train["tp_score"].quantile(.75),2,train["tp_per"])    

test['tp_per'] = ''
test['tp_per'] = np.where(test["tp_score"]<test["tp_score"].quantile(.25),0,test["tp_per"])
test['tp_per'] = np.where((test["tp_score"]>=test["tp_score"].quantile(.25)) & (test["tp_score"]<=test["tp_score"].quantile(.75)),1,test["tp_per"])
test['tp_per'] = np.where(test["tp_score"]>test["tp_score"].quantile(.75),2,test["tp_per"])    

train = train.astype({'tp_per':'int64'})
test = test.astype({'tp_per':'int64'})

train

,QaA,QaE,QbA,QbE,QcA,QcE,QdA,QdE,QeA,QeE,QfA,QfE,QgA,QgE,QhA,QhE,QiA,QiE,QjA,QjE,QkA,QkE,QlA,QlE,QmA,QmE,QnA,QnE,QoA,QoE,QpA,QpE,QqA,QqE,QrA,QrE,QsA,QsE,QtA,QtE,age_group,education,engnat,familysize,gender,hand,married,race,religion,tp01,tp02,tp03,tp04,tp05,tp06,tp07,tp08,tp09,tp10,urban,voted,wf_01,wf_02,wf_03,wr_01,wr_02,wr_03,wr_04,wr_05,wr_06,wr_07,wr_08,wr_09,wr_10,wr_11,wr_12,wr_13,mach_score,mach_per,tp_score,tp_per
index,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,3.0,363,4.0,1370,5.0,997,1.0,1024,2.0,1577,5.0,539,2.0,586,4.0,1095,5.0,1142,4.0,1287,4.0,883,4.0,851,2.0,851,5.0,816,2.0,579,2.0,924,2.0,366,2.0,876,2.0,633,1.0,1115,1,1,1,4,Female,1,2,0,0,2,2,2,1,2,1,7,4,4,3,1,2,0,0,0,0,1,0,0,1,0,1,1,0,1,0,1,1,3.05,1,2.8,1
1,5.0,647,5.0,1313,3.0,3387,5.0,2969,1.0,4320,3.0,2190,1.0,826,1.0,4082,5.0,1867,3.0,1264,5.0,2943,4.0,3927,1.0,4329,5.0,1828,1.0,1214,5.0,2414,5.0,1356,1.0,3039,4.0,4304,1.0,1346,1,3,2,3,Female,1,0,2,1,1,1,0,0,1,2,3,4,0,4,3,2,0,0,0,0,1,0,1,1,0,1,1,0,1,0,1,1,3.20,0,1.6,0
2,4.0,1623,1.0,1480,1.0,1021,4.0,3374,5.0,1333,1.0,531,4.0,1167,1.0,1016,3.0,2653,2.0,1569,5.0,998,5.0,2547,2.0,918,4.0,2153,2.0,1304,1.0,1131,5.0,937,4.0,1327,1.0,1170,1.0,1409,1,2,1,3,Male,1,1,0,0,2,3,1,5,3,4,2,6,1,3,2,1,0,0,1,1,1,0,1,1,0,1,1,1,1,0,1,1,2.80,0,3.0,2
3,3.0,504,3.0,2311,4.0,992,3.0,3245,1.0,357,2.0,1519,4.0,159,3.0,2275,5.0,2809,4.0,5614,3.0,3219,4.0,1296,4.0,9046,4.0,1216,4.0,1169,4.0,23868,3.0,581,4.0,8830,4.0,2392,5.0,1312,1,3,2,0,Female,1,0,2,1,2,4,1,1,1,3,1,3,1,3,3,1,0,0,0,0,1,0,0,0,0,0,1,0,1,0,1,1,3.55,1,2.0,0
4,1.0,927,1.0,707,5.0,556,2.0,1062,1.0,1014,2.0,628,1.0,991,1.0,1259,5.0,1153,5.0,1388,5.0,740,5.0,1181,4.0,547,2.0,575,1.0,754,4.0,1140,5.0,323,5.0,1070,1.0,583,2.0,1889,1,2,1,2,Male,1,1,0,0,1,1,1,6,0,2,0,6,2,6,1,1,0,1,0,1,1,0,1,1,1,1,1,0,1,1,1,1,2.90,1,2.5,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45527,2.0,1050,5.0,619,4.0,328,1.0,285,1.0,602,1.0,267,1.0,315,1.0,483,1.0,1016,4.0,278,4.0,611,5.0,215,4.0,388,1.0,299,2.0,1915,4.0,439,2.0,823,1.0,309,5.0,586,4.0,2252,0,1,1,1,Female,3,0,0,2,1,2,6,1,2,6,3,0,6,4,3,2,0,0,1,1,1,0,1,1,0,1,1,1,1,0,1,1,2.65,2,3.1,2
45528,2.0,581,3.0,1353,4.0,1164,1.0,798,3.0,1680,2.0,560,2.0,640,1.0,1415,2.0,4494,5.0,1392,2.0,2478,5.0,1268,3.0,843,2.0,1401,4.0,1524,4.0,719,2.0,754,3.0,1118,4.0,654,4.0,1197,1,1,1,2,Male,1,0,0,0,2,2,2,3,1,3,3,4,1,5,1,2,0,0,1,1,1,1,1,1,0,1,1,1,1,0,1,1,2.90,1,2.6,1
45529,4.0,593,1.0,857,1.0,1047,4.0,1515,5.0,1690,4.0,1253,5.0,1094,1.0,1283,5.0,2209,1.0,1764,5.0,1062,5.0,1489,1.0,1908,4.0,891,1.0,1298,1.0,1200,5.0,473,5.0,1779,3.0,1355,1.0,667,0,1,1,1,Female,1,0,0,2,4,4,2,4,0,1,0,2,1,6,2,1,0,0,0,1,1,0,1,1,0,1,1,0,1,0,1,1,3.10,0,2.4,1


In [12]:

# train['외향'] = (train['tp01'] + (7 - train['tp06']))/2
# train['감성'] = ((7 - train['tp02']) + train['tp07'])/2
# train['개방'] = (train['tp03'] + (7 - train['tp08']))/2
# train['친화'] = ((7 - train['tp04']) + train['tp09'])/2
# train['성실'] = (train['tp05'] + (7 - train['tp10']))/2

# train['외향_'] = (train['외향'] > 3.44) * 1
# train['감성_'] = (train['감성'] > 3.83) * 1
# train['개방_'] = (train['개방'] > 4.38) * 1
# train['친화_'] = (train['친화'] > 4.23) * 1
# train['성실_'] = (train['성실'] > 4.40) * 1

train['M+T'] = train['mach_score'] + train['tp_score']
test['M+T'] = test['mach_score'] + test['tp_score']

train['MT_per'] = ''
train['MT_per'] = np.where(train["M+T"]<train["M+T"].quantile(.25),0,train["MT_per"])
train['MT_per'] = np.where((train["M+T"]>=train["M+T"].quantile(.25)) & (train["M+T"]<=train["M+T"].quantile(.75)),1,train["MT_per"])
train['MT_per'] = np.where(train["M+T"]>train["M+T"].quantile(.75),2,train["MT_per"])    

test['MT_per'] = ''
test['MT_per'] = np.where(test["M+T"]<test["M+T"].quantile(.25),0,test["MT_per"])
test['MT_per'] = np.where((test["M+T"]>=test["M+T"].quantile(.25)) & (test["M+T"]<=test["M+T"].quantile(.75)),1,test["MT_per"])
test['MT_per'] = np.where(test["M+T"]>test["M+T"].quantile(.75),2,test["MT_per"])    
train

,QaA,QaE,QbA,QbE,QcA,QcE,QdA,QdE,QeA,QeE,QfA,QfE,QgA,QgE,QhA,QhE,QiA,QiE,QjA,QjE,QkA,QkE,QlA,QlE,QmA,QmE,QnA,QnE,QoA,QoE,QpA,QpE,QqA,QqE,QrA,QrE,QsA,QsE,QtA,QtE,age_group,education,engnat,familysize,gender,hand,married,race,religion,tp01,tp02,tp03,tp04,tp05,tp06,tp07,tp08,tp09,tp10,urban,voted,wf_01,wf_02,wf_03,wr_01,wr_02,wr_03,wr_04,wr_05,wr_06,wr_07,wr_08,wr_09,wr_10,wr_11,wr_12,wr_13,mach_score,mach_per,tp_score,tp_per,M+T,MT_per
index,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,3.0,363,4.0,1370,5.0,997,1.0,1024,2.0,1577,5.0,539,2.0,586,4.0,1095,5.0,1142,4.0,1287,4.0,883,4.0,851,2.0,851,5.0,816,2.0,579,2.0,924,2.0,366,2.0,876,2.0,633,1.0,1115,1,1,1,4,Female,1,2,0,0,2,2,2,1,2,1,7,4,4,3,1,2,0,0,0,0,1,0,0,1,0,1,1,0,1,0,1,1,3.05,1,2.8,1,5.85,1
1,5.0,647,5.0,1313,3.0,3387,5.0,2969,1.0,4320,3.0,2190,1.0,826,1.0,4082,5.0,1867,3.0,1264,5.0,2943,4.0,3927,1.0,4329,5.0,1828,1.0,1214,5.0,2414,5.0,1356,1.0,3039,4.0,4304,1.0,1346,1,3,2,3,Female,1,0,2,1,1,1,0,0,1,2,3,4,0,4,3,2,0,0,0,0,1,0,1,1,0,1,1,0,1,0,1,1,3.20,0,1.6,0,4.80,0
2,4.0,1623,1.0,1480,1.0,1021,4.0,3374,5.0,1333,1.0,531,4.0,1167,1.0,1016,3.0,2653,2.0,1569,5.0,998,5.0,2547,2.0,918,4.0,2153,2.0,1304,1.0,1131,5.0,937,4.0,1327,1.0,1170,1.0,1409,1,2,1,3,Male,1,1,0,0,2,3,1,5,3,4,2,6,1,3,2,1,0,0,1,1,1,0,1,1,0,1,1,1,1,0,1,1,2.80,0,3.0,2,5.80,1
3,3.0,504,3.0,2311,4.0,992,3.0,3245,1.0,357,2.0,1519,4.0,159,3.0,2275,5.0,2809,4.0,5614,3.0,3219,4.0,1296,4.0,9046,4.0,1216,4.0,1169,4.0,23868,3.0,581,4.0,8830,4.0,2392,5.0,1312,1,3,2,0,Female,1,0,2,1,2,4,1,1,1,3,1,3,1,3,3,1,0,0,0,0,1,0,0,0,0,0,1,0,1,0,1,1,3.55,1,2.0,0,5.55,1
4,1.0,927,1.0,707,5.0,556,2.0,1062,1.0,1014,2.0,628,1.0,991,1.0,1259,5.0,1153,5.0,1388,5.0,740,5.0,1181,4.0,547,2.0,575,1.0,754,4.0,1140,5.0,323,5.0,1070,1.0,583,2.0,1889,1,2,1,2,Male,1,1,0,0,1,1,1,6,0,2,0,6,2,6,1,1,0,1,0,1,1,0,1,1,1,1,1,0,1,1,1,1,2.90,1,2.5,1,5.40,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45527,2.0,1050,5.0,619,4.0,328,1.0,285,1.0,602,1.0,267,1.0,315,1.0,483,1.0,1016,4.0,278,4.0,611,5.0,215,4.0,388,1.0,299,2.0,1915,4.0,439,2.0,823,1.0,309,5.0,586,4.0,2252,0,1,1,1,Female,3,0,0,2,1,2,6,1,2,6,3,0,6,4,3,2,0,0,1,1,1,0,1,1,0,1,1,1,1,0,1,1,2.65,2,3.1,2,5.75,1
45528,2.0,581,3.0,1353,4.0,1164,1.0,798,3.0,1680,2.0,560,2.0,640,1.0,1415,2.0,4494,5.0,1392,2.0,2478,5.0,1268,3.0,843,2.0,1401,4.0,1524,4.0,719,2.0,754,3.0,1118,4.0,654,4.0,1197,1,1,1,2,Male,1,0,0,0,2,2,2,3,1,3,3,4,1,5,1,2,0,0,1,1,1,1,1,1,0,1,1,1,1,0,1,1,2.90,1,2.6,1,5.50,1
45529,4.0,593,1.0,857,1.0,1047,4.0,1515,5.0,1690,4.0,1253,5.0,1094,1.0,1283,5.0,2209,1.0,1764,5.0,1062,5.0,1489,1.0,1908,4.0,891,1.0,1298,1.0,1200,5.0,473,5.0,1779,3.0,1355,1.0,667,0,1,1,1,Female,1,0,0,2,4,4,2,4,0,1,0,2,1,6,2,1,0,0,0,1,1,0,1,1,0,1,1,0,1,0,1,1,3.10,0,2.4,1,5.50,1


In [13]:
def Filter(x):
    if x<1:
        return 0
    elif x<2:
        return 0
    elif x<6:
        return 1
    else:
        return 2
    
cols=['tp01','tp02','tp03','tp04','tp05','tp06','tp07','tp08','tp09','tp10']

def get_tp(df):
    for col in cols:
        df[col]=df[col].apply(Filter)
    return df

train=get_tp(train)
test=get_tp(test)

In [14]:
df = train.loc[:,['wf_01','wf_02','wf_03']]
df_list= df.values.tolist()
wf_new = []
for i in df_list:
    if i.count(1) >= 1 :
        wf_new.append(0)
    else :
        wf_new.append(1)
        
train["wf_new"]=wf_new

df = test.loc[:,['wf_01','wf_02','wf_03']]
df_list= df.values.tolist()
wf_new = []
for i in df_list:
    if i.count(1) >= 1 :
        wf_new.append(0)
    else :
        wf_new.append(1)
        
test["wf_new"]=wf_new

In [15]:
dr = train.loc[:,['wr_03','wr_06','wr_09','wr_11']]
dr_list= dr.values.tolist()
wr_new = []
for i in dr_list:
    if i.count(1) >= 2 :
        wr_new.append(1)
    else :
        wr_new.append(0)
train["wr_new"]=wr_new     

dr = test.loc[:,['wr_03','wr_06','wr_09','wr_11']]
dr_list= dr.values.tolist()
wr_new = []
for i in dr_list:
    if i.count(1) >= 2 :
        wr_new.append(1)
    else :
        wr_new.append(0)
test["wr_new"]=wr_new     

In [16]:
# 15인 초과 가구 2인가구로 변경
train['familysize'] = np.where(train['familysize'] > 15,2,train['familysize'])
train['familysize'] = np.where(train['familysize'] >=5,5,train['familysize'])

test['familysize'] = np.where(test['familysize'] > 15,2,test['familysize'])
test['familysize'] = np.where(test['familysize'] >=5,5,test['familysize'])


In [17]:
train['voted']=train['voted'].map({2:1,1:0})

In [18]:
del train["hand"]

del test["hand"]

In [19]:
train_x = train.drop('voted', axis = 1)
train_y = train['voted']

In [20]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train_x,train_y, test_size=0.2, random_state=123)

In [21]:
from catboost import CatBoostClassifier

clf = CatBoostClassifier(
    iterations=5, 
    learning_rate=0.1, 
    #loss_function='CrossEntropy'
)


clf.fit(X_train, y_train, 
#         cat_features=cat_features, 
#         eval_set=(X_val, y_val), 
#         verbose=False
)

print('CatBoost model is fitted: ' + str(clf.is_fitted()))
print('CatBoost model parameters:')
print(clf.get_params())

CatBoostError: Bad value for num_feature[non_default_doc_idx=0,feature_idx=44]="Male": Cannot convert 'b'Male'' to float

In [ ]:
y_pred_pro = clf.predict_proba(data=X_test)

In [ ]:
y_pred_pro

In [ ]:
y_pred_cat = clf.predict(data=X_test)

In [ ]:
y_pred_cat

In [ ]:
accuracy_score(y_test,y_pred_cat)

In [ ]:
y_pred_cat_1 = clf.predict(test)

In [ ]:
y_pred_cat_1

In [ ]:
y_pred_cat_1 = y_pred_cat_1+1

In [ ]:
submission['voted']=y_pred_cat_1
submission=submission.reset_index()

In [ ]:
submission  ##결과값 12인지 확인

In [ ]:
submission.to_csv('C:\\Users\Hyelin\\bigdata\\FinalProject\\TeamProject\\open data\\sample_submission_cat_final.csv',index=False)